In [1]:
#!pip install transformers

In [2]:
#!pip install sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import scipy
import numpy as np
import random
import sys
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

2021-12-08 11:43:43.173519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/apps/cuDNN/7.6.5-cuda-10.2/lib64
2021-12-08 11:43:43.173556: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
files = os.listdir("./From-ScisummNet-2019")
files

['P83-1020',
 'W02-1039',
 'P96-1021',
 'N06-1020',
 'C86-1016',
 'C92-2070',
 'D08-1082',
 'J01-2002',
 'C02-1114',
 'P85-1018',
 'P07-1032',
 'W00-1427',
 'C04-1111',
 'J97-1002',
 'W04-3237',
 'E87-1002',
 'P93-1041',
 'P90-1034',
 'P98-1035',
 'W02-0505',
 'P13-1045',
 'A00-2026',
 'J03-4004',
 'N03-1003',
 'C90-2067',
 'P84-1075',
 'W02-0908',
 'M95-1005',
 'P07-1028',
 'C00-2136',
 'P01-1019',
 'N10-1056',
 'J81-4003',
 'P93-1024',
 'C90-3030',
 'W02-2024',
 'P98-1013',
 'P03-1019',
 'P07-1065',
 'I05-3017',
 'P07-1056',
 'W03-0419',
 'W00-0726',
 'A00-2031',
 'D07-1090',
 'P02-1040',
 'P05-1011',
 'P02-1043',
 'A00-2009',
 'H94-1046',
 'C00-1044',
 'P11-1138',
 'C02-1011',
 'C96-1055',
 'N06-2015',
 'N03-1024',
 'P08-1101',
 'P96-1025',
 'P02-1019',
 'P09-1088',
 'C94-1042',
 'P99-1068',
 'N07-1018',
 'D07-1043',
 'J93-1007',
 'C08-1022',
 'P01-1030',
 'J01-2001',
 'N01-1008',
 'P05-3026',
 'P96-1024',
 'A00-2019',
 'E06-1042',
 'N06-1014',
 'P03-1011',
 'P11-1016',
 'C92-2082',

In [3]:
train_end = int(len(files)*0.7)
# val_end = int(len(files)*0.1)+train_end
train_docs = files[0:train_end]
# val_docs = files[train_end:val_end]
test_docs = files[train_end:len(files)]

In [5]:
def preprocess(example_sent):
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(example_sent.lower())

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [w for w in filtered_sentence if w.isalpha()]
#     print(filtered_sentence)
    new = " " 
    a = new.join(filtered_sentence)
    return a

def get_dataset(files):
    pairs_lis = []
    new_corpus_prev = None
    for z,f in enumerate(files):
        try:
            citants = pd.read_json("From-ScisummNet-2019/"+str(f)+"/citing_sentences.json")
            citants = citants[['citance_No','clean_text']]
            queries = list(citants['clean_text'])
            cite_no = list(citants.citance_No)
            tree = ET.parse("From-ScisummNet-2019/"+f+"/Reference_XML/"+f+".xml")
            root = tree.getroot()
            final1=[]
            
            total = len(root)
            for a in root:
                for b in a:
                    final1.append(b.text)
                    

            d={'col1':final1}
            rp = pd.DataFrame(data=d)
            ann = "From-ScisummNet-2019/"+f+"/annotation/"+f+".ann.txt"
            with open(ann,"r") as f:
                data = f.read()
            idx = {'Citance Number':0,'Reference Article':1,'Citing Article':2,'Citation Marker Offset':3,'Citation Marker':4,'Citation Offset':5,'Reference Offset':7}
            gt = {}
            lines = data.split('\n')
            for line in lines:
                lis = line.split('|')
                if len(lis)==11:
                    cit_no = lis[0].split(':')[1]
                    ref_off = lis[7].split(':')[1].strip()
                    import ast
                    ref_off = ast.literal_eval(ref_off)
                    ref_off = list(map(int, ref_off))
                    gt[int(cit_no)] = ref_off 
            corpus = rp.col1
            new_corpus = corpus.apply(lambda x: preprocess(x))

            q_lis = []
            for q in queries:
                a = preprocess(q)
                if len(a)>1:
                    q_lis.append(a)
            del queries

            for i in range(len(q_lis)):
                if cite_no[i] in gt:
                    for j in gt[cite_no[i]]:
                        if int(j) < len(new_corpus):
                            pairs_lis.append(InputExample(texts=[q_lis[i],new_corpus[int(j)]],label=1.0)) #positive pairs
                            pairs_lis.append(InputExample(texts=[q_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.4)) #negative pairs
                            pairs_lis.append(InputExample(texts=[q_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.4))
                            pairs_lis.append(InputExample(texts=[q_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.4))
            if (z!=0 and len(new_corpus)!=0 and len(new_corpus_prev)!=0):
                pairs_lis.append(InputExample(texts = [new_corpus_prev[random.randint(0,len(new_corpus_prev)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))

            new_corpus_prev = new_corpus
        except Exception as e:
            print("eror in ",f)
            new_corpus_prev = None
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            prev_corpus = []
        
    return pairs_lis

    
train_data = get_dataset(train_docs)
# val_data = get_dataset(val_docs)
test_data = get_dataset(test_docs)

In [6]:
len(train_data)

65277

In [7]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=2)
# val_dataloader = DataLoader(val_data, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_data, shuffle=True, batch_size=2)
                

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')
train_loss = losses.CosineSimilarityLoss(model)
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32639 [00:00<?, ?it/s]

In [ ]:
import torch
torch.save(model,'bert_task1_2019.pth')

In [ ]:
# model = torch.load('distilbert_fine_tuned.pth')

In [ ]:
# !pip install git+https://github.com/tagucci/pythonrouge.git

In [ ]:
# from ignite.metrics import Rouge  #https://pypi.org/project/pytorch-ignite/
from rouge_score import rouge_scorer  #https://pypi.org/project/rouge-score/
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
from pythonrouge.pythonrouge import Pythonrouge


def get_matching_sentences(model,corpus,queries,gt,cite_no):
    #f1 score will be same as precision and recall in our case: since documents
    
    # Get a vector for each headline (sentence) in the corpus
    corpus_embeddings = model.encode(corpus)
    corpus_embeddings = corpus_embeddings/np.linalg.norm(corpus_embeddings,axis=0).reshape(-1)
    # Define search queries and embed them to vectors as well

    query_embeddings = model.encode(queries)
    # For each search term return 3 closest sentences
    closest_n = 3
    total_nums_correct = 0
    total_retrieved = 0
    total_relevent = 0
    rouge1 = []
    rouge2 = []
    rouge_su4 = []
    for i in range(len(queries)):
        query, query_embedding  = queries[i], query_embeddings[i]
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])

#         print("\n\n======================\n\n")
#         print("Query:", query)
        indexes = results[0:closest_n]
        top_n = []
        for l,k in indexes:
            top_n.append(l)
        if cite_no[i] in gt:
            nums_correct = len(np.intersect1d(top_n,gt[cite_no[i]]))
            total_nums_correct += nums_correct
            total_relevent += len(gt[cite_no[i]])
            total_retrieved += 3
        for idx, distance in results[0:closest_n]:
            scores = scorer.score(corpus[idx].strip(), query)
            rouge = Pythonrouge(summary_file_exist=False,
                    summary=[[corpus[idx].strip()]], reference=[[[query]]],
                    n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                    recall_only=True, stemming=True, stopwords=True,
                    word_level=True, length_limit=True, length=50,
                    use_cf=False, cf=95, scoring_formula='average',
                    resampling=True, samples=1000, favor=True, p=0.5)
    
            score = rouge.calc_score()
            rouge1.append(score['ROUGE-1'])
            rouge2.append(score['ROUGE-2'])
            rouge_su4.append(score['ROUGE-SU4'])

            
    return total_nums_correct, total_relevent,total_retrieved, np.mean(rouge1), np.mean(rouge2), np.mean(rouge_su4)
    
                    
#             print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.environ['TOKENIZERS_PARALLELISM']= '0'

def evaluate(model,files):
    gt_big = {}
    corpus_big = {}
    queries_big = {}
    cite_no_big = {}
    rouge1_lis = []
    rouge2_lis = []
    rouge_su4_lis = []
    tp_big = 0
    tot_relevent = 0
    tot_retrieved = 0
    for z,f in enumerate(files):
        try:
            citants = pd.read_json("From-ScisummNet-2019/"+str(f)+"/citing_sentences.json")
            citants = citants[['citance_No','clean_text']]
            queries = list(citants['clean_text'])
            cite_no = list(citants.citance_No)
            tree = ET.parse("From-ScisummNet-2019/"+f+"/Reference_XML/"+f+".xml")
            root = tree.getroot()
            final1=[]
            final2=[]


            i = 0
            total = len(root)
            for a in root:
                for b in a:
                    final1.append(b.text)
                    if i == 0:
                        final2.append("Abstract")
                    if i == 1:
                        final2.append("Introduction")
                    elif i < total-2:
                        final2.append("Experiment/Discussion")
                    if i == total-2 or i == total-1:
                        final2.append("Results/Conclusion")
                    if i == total:
                        final2.append("Acknowledgment")
                i = i+1

            d={'col1':final1,'col2':final2}
            rp = pd.DataFrame(data=d)
            ann = "From-ScisummNet-2019/"+f+"/annotation/"+f+".ann.txt"
            with open(ann,"r") as f:
                data = f.read()
            idx = {'Citance Number':0,'Reference Article':1,'Citing Article':2,'Citation Marker Offset':3,'Citation Marker':4,'Citation Offset':5,'Reference Offset':7}
            gt = {}
            lines = data.split('\n')
            for line in lines:
                lis = line.split('|')
                if len(lis)==11:
                    cit_no = lis[0].split(':')[1]
                    ref_off = lis[7].split(':')[1].strip()
                    import ast
                    ref_off = ast.literal_eval(ref_off)
                    ref_off = list(map(int, ref_off))
                    gt[int(cit_no)] = ref_off 
            corpus = rp.col1
            new_corpus = corpus.apply(lambda x: preprocess(x))

            q_lis = []
            for q in queries:
                a = preprocess(q)
                if len(a)>1:
                    q_lis.append(a)

            if len(new_corpus)==0:
                continue

            tp, tot_rele,tot_retrieved, rouge1, rouge2, rouge_su4 = get_matching_sentences(model,new_corpus,q_lis,gt,cite_no)
            tp_big += tp
            tot_relevent += tot_rele
            tot_retrieved += tot_retrieved

            rouge1_lis.append(rouge1)
            rouge2_lis.append(rouge2)
            rouge_su4_lis.append(rouge_su4)
        except Exception as e:
            print(e, f)
        
    recall = tp_big/tot_relevent
    precision = tp_big/tot_retrieved
    f1 = 2*recall*precision/(recall+precision)
    return recall,precision,f1, np.mean(rouge1_lis), np.mean(rouge2_lis), np.mean(rouge_su4_lis)
    
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,train_docs)   
print("metrics obtained in train: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

In [ ]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,val_docs)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
#pre-trained model
model_pt = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model_pt,val_docs)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))